In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
url = 'https://data.tainan.gov.tw/api/3/action/datastore_search?resource_id=6fb38177-3c0d-48ce-8aa9-95fcd7e32c0c'
#url = 'https://data.tainan.gov.tw/dataset/site/resource/6fb38177-3c0d-48ce-8aa9-95fcd7e32c0c'
#fileobj = urllib.request.urlopen(url)
r = requests.get(url)
jdata = r.json()

In [3]:
jdata

{'help': 'https://data.tainan.gov.tw/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'resource_id': '6fb38177-3c0d-48ce-8aa9-95fcd7e32c0c',
  'fields': [{'type': 'int', 'id': '_id'},
   {'type': 'text', 'id': '遺址名稱'},
   {'type': 'text', 'id': '行政隸屬'},
   {'type': 'text', 'id': '地理環境'},
   {'type': 'text', 'id': '海拔高度'},
   {'type': 'text', 'id': '所屬水系'},
   {'type': 'text', 'id': '遺址狀態簡述'},
   {'type': 'text', 'id': '遺址狀態分布'},
   {'type': 'text', 'id': '遺址面積'},
   {'type': 'text', 'id': '文化類型'},
   {'type': 'text', 'id': '遺址年代'},
   {'type': 'text', 'id': '遺址代表意義與文化意義'}],
  'records_format': 'objects',
  'records': [{'_id': 1,
    '遺址名稱': '照興遺址',
    '行政隸屬': '臺南市楠西區照興里',
    '地理環境': '嘉南平原－曾文溪沖積平原',
    '海拔高度': '95~100',
    '所屬水系': '曾文溪（曾文溪／曾文溪集水區）',
    '遺址狀態簡述': '遺址位於照興聚落之東北側約50公尺處，曾文溪及南投埤坑之北側平台上，七里西農園包含於遺址的範圍之內。',
    '遺址狀態分布': '遺址範圍內地表即為文化層，因果園耕作擾出許多橙色系夾砂陶片及黑色系陶片，均勻分布於地表面上。',
    '遺址面積': '長寬大致500m×300m，面積約120000 ㎡',
    '文化類型':

In [4]:
def get_target_value(key, dic, tmp_list):
    """
    :param key: 目标key值
    :param dic: JSON数据
    :param tmp_list: 用于存储获取的数据
    :return: list
    """
    if not isinstance(dic, dict) or not isinstance(tmp_list, list):  # 对传入数据进行格式校验
        return 'argv[1] not an dict or argv[-1] not an list '

    if key in dic.keys():
        tmp_list.append(dic[key])  # 传入数据存在则存入tmp_list

    for value in dic.values():  # 传入数据不符合则对其value值进行遍历
        if isinstance(value, dict):
            get_target_value(key, value, tmp_list)  # 传入数据的value值是字典，则直接调用自身
        elif isinstance(value, (list, tuple)):
            _get_value(key, value, tmp_list)  # 传入数据的value值是列表或者元组，则调用_get_value


    return tmp_list


def _get_value(key, val, tmp_list):
    for val_ in val:
        if isinstance(val_, dict):  
            get_target_value(key, val_, tmp_list)  # 传入数据的value值是字典，则调用get_target_value
        elif isinstance(val_, (list, tuple)):
            _get_value(key, val_, tmp_list)   # 传入数据的value值是列表或者元组，则调用自身

In [5]:
遺址名稱 = get_target_value('遺址名稱', jdata, [])
行政隸屬 = get_target_value('行政隸屬', jdata, [])
遺址狀態簡述 = get_target_value('遺址狀態簡述', jdata, [])
遺址狀態分布 = get_target_value('遺址狀態分布', jdata, [])
文化類型 = get_target_value('文化類型', jdata, [])
遺址面積 = get_target_value('遺址面積', jdata, [])
序號 = get_target_value('_id', jdata, [])

In [6]:
df1 = pd.DataFrame(遺址名稱, columns=['遺址名稱'])
df2 = pd.DataFrame(行政隸屬, columns=['行政隸屬'])
df3 = pd.DataFrame(遺址狀態簡述, columns=['遺址狀態簡述'])
df4 = pd.DataFrame(遺址狀態分布, columns=['遺址狀態分布'])
df5 = pd.DataFrame(文化類型, columns=['文化類型'])
df6 = pd.DataFrame(遺址面積, columns=['遺址面積'])

In [7]:
list_comb = [df1, df2, df3, df4, df5, df6]

In [8]:
df_comb = pd.concat(list_comb, axis=1)

In [9]:
df_comb

,遺址名稱,行政隸屬,遺址狀態簡述,遺址狀態分布,文化類型,遺址面積
0,照興遺址,臺南市楠西區照興里,遺址位於照興聚落之東北側約50公尺處，曾文溪及南投埤坑之北側平台上，七里西農園包含於遺址的範...,遺址範圍內地表即為文化層，因果園耕作擾出許多橙色系夾砂陶片及黑色系陶片，均勻分布於地表面上。,「鹿陶類型」文化,長寬大致500m×300m，面積約120000 ㎡
1,濺尿仔遺物出土地點,臺南市楠西區照興里,遺址位於興北聚落之東北側約650公尺處，其東側200公尺為曾文水庫管理局收費站，而曾文溪由北...,遺物於小產業道路之坡面平台上，僅見少許細碎橙色夾砂陶及陶把，周邊可見早期開發時留下之大型石牆...,「鹿陶類型」文化,點狀分布
2,興北遺址,臺南市楠西區照興里,遺址位於興北聚落之東側約400公尺處，楠西曾文一橋之東北側，由174縣道55.5公里處，為曾...,由於遺址範圍內為植被區，其遺址於丘頂上緣風化嚴重，故僅於斷面可看見少量夾砂陶片。,「鹿陶類型」文化,點狀分布
3,盧芝潭遺址,臺南市楠西區密枝里,遺址位於盧芝潭聚落內，其西側為曾文溪流經，東側為密枝溪流經，且隔密枝溪與「楠西‧樣子坑遺址」...,遺址為雙溪匯流之高位河階，文化層可能於地表枝淺層，又因開墾果園擾動而導致分布於地表之橙色夾砂...,「鹿陶類型」文化,點狀分布
4,兩奶山遺址,臺南市楠西區照興里,遺址位於興北聚落東南側約800公尺處，由萬佛寺前的產業道路進入曾文溪岸，曾文溪於遺址東南側流...,曾文溪之兩奶山北側岸沿緩坡地，現有被開墾梯田約1.5甲地，另梯田的高落差斷面，有密度極高之夾...,鹿陶類型(臧振華等 1994：1124-LNS-1)、「鹿陶類型」文化,長寬大致150m×350m，面積約45000 ㎡
5,密枝遺址,臺南市楠西區密枝里,遺址位於樣子坑聚落之西南側700公尺處，其南側為楠西區第一公墓，而密枝溪及曾文溪匯流經遺址之...,曾文溪南岸往密枝里(樣子坑)前，於楊桃樹邊的水溝二側，發現部分夾砂橙色陶片，但其附近台地夾砂...,「鹿陶類型」文化,點狀分布
6,楠西遺址,臺南市楠西區楠西里,遺址位於楠西聚落範圍內之西側，民族路和中華路交會處即為遺址所在位置，北極殿後方與東北側「楠西...,曾文溪東側沖積台地中位河階地，沿岸有少許橙色陶片，未見文化層，四周大多已覆蓋建築物，露土處不多。,「鹿陶類型」文化,點狀分布
7,楠西國中遺址,臺南市楠西區楠西里,遺址位於楠西聚落之北側，而曾文溪流經遺址之西北側，楠西國中操場即為遺址所在，其西南側400公...,曾文溪的南側中位河階台地，以楠西國中東側及中興路之二側，可見到夾砂陶片分布。,「鹿陶類型」文化,長寬大致350m×100m，面積約28000 ㎡
8,楠西國小遺址,臺南市楠西區楠西里,遺址位於楠西聚落內之南側，西北側可見楠西國小，為油車溪之支流流經，而灣丘溪流經其東南側，西北...,由中正路與信義路交會處即是遺址範圍，為中正路319號前，僅於西瓜田中發現細碎零星之夾砂陶片。,「鹿陶類型」文化,點狀分布
9,楠西‧樣子坑遺址,臺南市楠西區密枝里,遺址位於樣子坑聚落內北側約50公尺處，密枝溪由東向西南流經本遺址與曾文溪匯流而下，地是由東南...,遺址範圍內因果園開挖水溝而擾出不少文化遺物，可見包括黑色夾砂陶罐形器口緣及橙色夾細砂陶片，除...,丘陵地區遺址群,長寬大致50m×50m，面積約2000 ㎡


In [10]:
writer = pd.ExcelWriter('考古遺址_楠西區.xlsx')

df_comb.to_excel(writer, sheet_name='楠西區')
writer.save()